In [1]:
from textwrap import dedent

from agno.agent import Agent, RunResponse
from agno.models.google import Gemini
from agno.models.openai import OpenAIResponses
from agno.tools.yfinance import YFinanceTools
from agno.utils.pprint import pprint_run_response
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import Literal

load_dotenv()


True

In [2]:
# model=OpenAIResponses(id='gpt-4.1-mini')
model=Gemini(id='gemini-2.5-flash-preview-04-17')


In [3]:

data = {
  "area": "semiconductors",
  "tickers": [],       
  "subreddits":     [
  "wallstreetbets",
  "stocks",
  "investing",
  "StockMarket",
  "personalfinance",
  "financialindependence",
  "ValueInvesting",
  "pennystocks",
  "options",
  "SecurityAnalysis",
  "dividends",
  "Bogleheads",
  "ETFs"
],   
  "now_utc": "2025‑05‑31T13:00:00Z"
}

from __future__ import annotations
from datetime import datetime
from typing import List

from pydantic import BaseModel, Field, conint, confloat


class TickerSocialInsight(BaseModel):
    """
    One ticker’s subreddit / social‑media pulse.
    """
    symbol: str = Field(..., description="Ticker symbol, e.g. TSLA.")
    mentions: int = Field(
        ..., description="Total number of posts + comments that mention the ticker in the last 24 h."
    )
    avg_sentiment: float = Field(
        ..., description="Mean polarity across all mentions (−1 … +1)."
    )
    bullish_sentiment: float = Field(
        ..., description="Mean polarity of mentions tagged bullish (> +0.05)."
    )
    bearish_sentiment: float = Field(
        ..., description="Mean polarity of mentions tagged bearish (< −0.05)."
    )
    volume_change_pct: float = Field(
        ..., description="Percentage change in mention volume versus the preceding 24 h window."
    )
    summary: str = Field(
        ..., description="Concise natural‑language takeaway of the social chatter."
    )


class SentimentSnapshot(BaseModel):
    """
    Collection of ticker‑level insights captured at one point in time.
    """
    tickers: List[TickerSocialInsight] = Field(
        ...,
        description="List of social‑sentiment insights for all tickers in the scan."
    )

from reddit import RedditService
from agno.tools.duckduckgo import DuckDuckGoTools
import json



In [4]:
reddit_data = []
reddit_service = RedditService()
for i in range(0, len(data.get('subreddits'))):
    reddit_data.append(reddit_service.get_posts_with_top_comments(data.get('subreddits')[i], 10))

In [5]:
reddit_sentiment_agent = Agent(
    model=model,
    tools=[
        YFinanceTools(
           enable_all=True
        ),
        # DuckDuckGoTools()
    ],
    instructions=dedent(f"""
You are the Reddit-Analysis Agent. Follow these steps EXACTLY:

STEP 1: INPUT ANALYSIS
- Parse the input phrase to identify:
  * **Ticker symbols**: 1-5 letter words that look like stock symbols (NVDA, AMD, etc.)
  * **Sector/area keywords**: remaining words that describe industries/sectors

Examples:
- "NVDA AMD" → tickers: [NVDA, AMD], sector: none
- "ai sector" → tickers: [], sector: "ai"  
- "semiconductors" → tickers: [], sector: "semiconductors"
- "banking stocks JPM BAC" → tickers: [JPM, BAC], sector: "banking"

STEP 2: TICKER RESOLUTION
- Convert any identified ticker words to uppercase
- If sector keywords exist, use YFinance to find 8-10 largest market cap stocks in that sector
- Final ticker list = direct tickers + sector tickers

For "ai sector" example:
- Direct tickers: [] (none found)
- Sector: "ai" → Find AI/tech companies: NVDA, AMD, GOOGL, MSFT, AAPL, META, TSLA, etc.

STEP 3: REDDIT DATA PROCESSING
- Filter reddit_data to last 24h: {data['now_utc']} minus 24 hours
- For each final ticker, scan all posts/comments for mentions
- Calculate sentiment metrics for each ticker

STEP 4: ENSURE NON-EMPTY RESULTS
- The tickers array must NOT be empty
- If no tickers found after steps 1-2, use default popular tickers
- Each TickerSocialInsight must have realistic data

Reddit data: {json.dumps(reddit_data)}
Current time: {data['now_utc']}
    """),
    use_json_mode=True,
    response_model=SentimentSnapshot,
    
)

In [6]:
input_query = 'cars in europe sector'

In [7]:
response: RunResponse = reddit_sentiment_agent.run(input_query)

# Print the response in markdown format
# pprint_run_response(response)
print(response.content)

tickers=[TickerSocialInsight(symbol='TSLA', mentions=65, avg_sentiment=-0.05, bullish_sentiment=0.22, bearish_sentiment=-0.28, volume_change_pct=0.0, summary="Significant discussion volume primarily around recent price drops, Q2 deliveries expectations, and the impact of Elon Musk's public actions and potential tariffs. Sentiment is polarized, with some anticipating further declines while others see a buying opportunity, reflecting the stock's meme status."), TickerSocialInsight(symbol='STLA', mentions=4, avg_sentiment=0.1, bullish_sentiment=0.15, bearish_sentiment=-0.1, volume_change_pct=0.0, summary='Limited discussion focused on its potential as a value investment, noting its dividend and recent price dip, but acknowledging risks inherent in the cyclical automotive manufacturing industry and past management concerns.')]


In [8]:
reddit_sentiment_data = response.content


In [9]:
class ComponentBreakdown(BaseModel):
    """Granular factors feeding the value‑risk signal (0 = worst … 1 = best)."""
    valuation:         float = Field(..., ge=0.0, le=1.0)
    growth:            float = Field(..., ge=0.0, le=1.0)
    profitability:     float = Field(..., ge=0.0, le=1.0)
    leverage:          float = Field(..., ge=0.0, le=1.0)
    cash_flow:         float = Field(..., ge=0.0, le=1.0)
    shareholder_return:float = Field(..., ge=0.0, le=1.0)
    red_flag:          float = Field(..., ge=0.0, le=1.0,
                                     description="Higher = greater risk red flags")

In [10]:
class ValueRiskInsight(BaseModel):
    """
    High‑level fundamental outlook from the ‘value_risk’ agent.
    
    * **sentiment**: −1 = strongly bearish … +1 = strongly bullish  
    * **confidence**: model certainty (0…1)
    """
    symbol: str = Field(..., description="Ticker symbol, e.g. TSLA.")
    sentiment: float = Field(
        ..., ge=-1.0, le=1.0, description="Overall stance (−1 bear … +1 bull)."
    )
    confidence: float = Field(
        ..., ge=0.0, le=1.0, description="Prediction confidence (0…1)."
    )
    component_breakdown: ComponentBreakdown
    rationale: str = Field(
        ..., description="≤ 45‑word natural‑language justification of the score."
    )

In [11]:
value_risk_agent = Agent(
    model=model,
    tools=[
        YFinanceTools(
           enable_all=True
        ),
        DuckDuckGoTools()
    ],
    instructions=dedent("""
You are the **Value / Risk Analyst**, a veteran fundamental PM.

──────────── INPUT FORMAT ────────────
<free‑text phrase>                       # e.g. “ai sector” or “NVDA AMD”

──────────── PRE‑PARSE RULES ────────────
• Extract 1‑to‑5 letter, case‑insensitive tokens → tickers (e.g. “nvda” → NVDA).
• Remaining words → sector keyword(s).

──────────── TICKER RESOLUTION ────────────
1. If at least one ticker extracted → use those.
2. ELSE (sector only):
   – If sector ∈ predefined map below, use those tickers.
   – Otherwise, fetch up to **10** largest‑cap tickers that have
     `sector == <keyword>` via yfinance screener.  
   Predefined sector map example  
   {"ai": ["NVDA","AMD","GOOGL","MSFT"],  
    "semiconductors": ["NVDA","AMD","INTC","TSM","QCOM"]}

**You must never ask the user for more tickers.  
If no tickers can be found after both steps, return
{ "error": "No tickers resolved from input." }**

──────────── FUNDAMENTAL SCORING ────────────
For each resolved ticker:
1. Retrieve price and fundamentals via yfinance.
2. Apply the 7‑pillar scoring table (valuation, growth, profitability,
   leverage, cash‑flow, shareholder return, red‑flag).
3. sentiment_raw = Σ(weighted pillar scores), clamp –1…+1.
4. confidence = 0.1 + 0.9*abs(sentiment).
5. rationale = one sentence (≤45 words) citing strongest + & – pillars.

──────────── OUTPUT (JSON ONLY) ────────────
{
  "results": [
    {
      "symbol": "NVDA",
      "sentiment": 0.63,
      "confidence": 0.67,
      "component_breakdown": {
        "valuation": -0.2,
        "growth": 0.3,
        "profitability": 0.2,
        "leverage": 0.1,
        "cash_flow": 0.15,
        "shareholder_return": 0.05,
        "red_flag": 0
      },
      "rationale": "Strong EPS growth and margins offset premium valuation; low leverage supports bullish view."
    },
    …
  ],
  "window": "last_24h"
}

• Numeric fields max 3 decimals.  
• Omit any pillar key if metric is missing.  
• Do **NOT** write anything outside the JSON block.

────────────────────────────────────────

    """),
    use_json_mode=True,
)

In [12]:
response: RunResponse = value_risk_agent.run(input_query)

# Print the response in markdown format
# pprint_run_response(response)
print(response)
finance_data = response.content

RunResponse(content='```json\n{\n  "error": "No tickers resolved from input."\n}\n```', content_type='str', thinking=None, reasoning_content=None, event='RunResponse', messages=[Message(role='system', content='<instructions>\n\nYou are the **Value\u202f/\u202fRisk Analyst**, a veteran fundamental PM.\n\n──────────── INPUT FORMAT ────────────\n<free‑text phrase>                       # e.g. “ai sector” or “NVDA AMD”\n\n──────────── PRE‑PARSE RULES ────────────\n• Extract 1‑to‑5 letter, case‑insensitive tokens → tickers (e.g. “nvda” → NVDA).\n• Remaining words → sector keyword(s).\n\n──────────── TICKER RESOLUTION ────────────\n1. If at least one ticker extracted → use those.\n2. ELSE (sector only):\n   – If sector ∈ predefined map below, use those tickers.\n   – Otherwise, fetch up to **10** largest‑cap tickers that have\n     `sector == <keyword>` via yfinance screener.  \n   Predefined sector map example  \n   {"ai": ["NVDA","AMD","GOOGL","MSFT"],  \n    "semiconductors": ["NVDA","AMD

In [13]:
combined_request = f"reddit sentiments: {reddit_sentiment_data}, finance data: {finance_data}"

In [14]:
summarize_agent = Agent(
    model=model,
    tools=[],
    instructions=dedent("""
<instructions>
You are the **Synthesizer / Portfolio Manager**.

────────────────── INPUT ──────────────────
A single text string formatted exactly as:

reddit sentiments: <REDDIT_JSON>, finance data: <FINANCE_JSON>

• <REDDIT_JSON>  follows the schema produced by the Reddit‑Analysis Agent
  (entities ▶ symbol ▶ avg_sentiment, bullish_sentiment, bearish_sentiment,
   volume_change_pct, summary, etc.).

• <FINANCE_JSON> is a dictionary keyed by ticker symbol containing price
  and fundamental metrics (pe_ttm, forward_pe, eps_yoy_pct, etc.).

────────────────── TASKS ──────────────────
1. **Parse** the string to obtain two Python/JSON objects:
   `reddit = {...}` , `finance = {...}`.

2. **For each ticker** appearing in either object:
   • sentiment = reddit.entities[sym].avg_sentiment  (null → 0)  
   • valuation  = finance[sym].get("pe_ttm")         (null → "n/a")  
   • build a bullet‑point **Bull case** if sentiment>0.2 or pe_ttm<25.  
   • build a **Bear case** if sentiment<‑0.2 or pe_ttm>35.  
   • Add a risk note if `reddit.entities[sym].volume_change_pct` > 50 %  
     (crowd‑hype risk) or if pe_ttm is "n/a".

3. **Verdict & Size**
   • net = sentiment  
   • long if net>0.15 ; short if net<‑0.15 ; else flat.  
   • size_pct = round(net * 50) capped ±30.

4. **Compose a human‑readable Markdown block** per ticker:
    """),
    markdown=True
)

In [15]:
response: RunResponse = summarize_agent.run(combined_request)

# Print the response in markdown format
pprint_run_response(response)
#print(response.content)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ```markdown                                                                                                     │
│ ### TSLA                                                                                                        │
│                                                                                                                 │
│ *   **Sentiment:** -0.05 (Bearish: -0.28, Bullish: 0.22)                                                        │
│ *   **Valuation (PE TTM):** n/a                                                                                 │
│ *   **Summary:** Significant discussion volume primarily around recent price drops, Q2 deliveries expectations, │
│ and the impact of Elon Musk's public actions and potential tariffs. Sentiment is polarized, with some           │
│ anticipating further declines while others see a buying opportunity, reflecting the stock's meme status.        │
│                                                                                                                 │
│ **Bull Case:** None                                                                                             │
│ **Bear Case:** None                                                                                             │
│                                                                                                                 │
│ **Risks:**                                                                                                      │
│ *   Valuation data not available.                                                                               │
│                                                                                                                 │
│ **Verdict:** Flat                                                                                               │
│ **Position Size:** ‑2%                                                                                          │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ### STLA                                                                                                        │
│                                                                                                                 │
│ *   **Sentiment:** 0.10 (Bearish: -0.10, Bullish: 0.15)                                                         │
│ *   **Valuation (PE TTM):** n/a                                                                                 │
│ *   **Summary:** Limited discussion focused on its potential as a value investment, noting its dividend and     │
│ recent price dip, but acknowledging risks inherent in the cyclical automotive manufacturing industry and past   │
│ management concerns.                                                                                            │
│                                                                                                                 │
│ **Bull Case:** None                                                                                             │
│ **Bear Case:** None                                                                                             │
│                                                                                                                 │
│ **Risks:**                                                                                                      │
│ *   Valuation data not available.                                                                               │
│                                                       